In [ ]:
import os
import numpy as np
import cv2
import datetime
from sklearn.model_selection import train_test_split
import tensorflow  as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Conv3D , MaxPooling3D , AvgPool3D , Input , Dense ,Flatten, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras import backend as K
import warnings
logger = tf.get_logger()
logger.setLevel('ERROR')
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import cv2
import os

import cv2
import numpy as np


def load_video(path, frame_count=40, target_size=(299, 299)):
    cap = cv2.VideoCapture(path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, frame_count, dtype=int)

    for frame_index in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, target_size)
            frame = frame / 255.0
            frames.append(frame)
        else:
            break

    cap.release()
    video_array = np.array(frames)

    return video_array


def create_dataset(directory, frame_count=40, target_size=(299, 299)):
    videos = []
    labels = []

    for folder in ["nofoul", "foul"]:
        folder_path = os.path.join(directory, folder)
        for filename in os.listdir(folder_path):
            video_path = os.path.join(folder_path, filename)
            video = load_video(video_path, frame_count=frame_count, target_size=target_size)
            if video.shape[0] == frame_count:
                videos.append(video)
                label = 1 if folder == "foul" else 0
                labels.append(label)

    return np.array(videos), np.array(labels)


In [ ]:
videos, labels = create_dataset("C:/Users/iturk/OneDrive/Desktop/Dataset_org")

MemoryError: Unable to allocate 21.1 GiB for an array with shape (264, 40, 299, 299, 3) and data type float64

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(videos, labels, test_size=0.15, random_state=42)

In [ ]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, AvgPool3D, Dense, Flatten, Add
from tensorflow.keras.models import Model
from ultralytics import YOLO

def yolo_pose(inputs):
  model = YOLO("yolov8n-pose.pt")  # load a pretained model
  poese = model.predict(source='Foul',save=True, imgsz=305,conf=0.2,save_txt=True , boxes = False )  # train the model
    return poese

def yolo_segmentation(inputs):
  model = YOLO("yolov8n-seg.pt")  # load a pretained model
  yolo_segmentation_output = model.predict(source='Foul',save=True, imgsz=305,conf=0.2,save_txt=True , boxes = False )  # train the model

    return yolo_segmentation_output

inputs = Input(shape=(305,305,3))
yolo_pose_output = yolo_pose(inputs)
yolo_pose_model = Model(inputs=inputs, outputs=yolo_pose_output)

inputs = Input(shape=(305,305,3))
yolo_segmentation_output = yolo_segmentation(inputs)
yolo_segmentation_model = Model(inputs=inputs, outputs=yolo_segmentation_output)

combined_output = Add()([yolo_pose_output, yolo_segmentation_output]) # addition +

In [ ]:
K.clear_session()
def inc_block(inputs):
    x0 = Conv3D(64,
                (1,1,1),
                activation='relu',
                padding="same")(inputs)
    x1 = Conv3D(64,
                (1, 1, 1),
                activation='relu',
                padding="same")(inputs)
    x1 = Conv3D(64,
                (3, 3, 3),
                activation='relu',
                padding="same")(x1)
    x2 = Conv3D(64,
                (1, 1, 1),
                activation='relu',
                padding="same")(inputs)
    x2 = Conv3D(64,
                (3, 3, 3),
                activation='relu',
                padding="same")(x2)
    x3 = MaxPooling3D((3, 3, 3),
                      (1,1,1),
                      padding="same")(inputs)
    x3 = Conv3D(64,
                (1,1,1),
                activation='relu',
                padding="same")(x3)
    outputs = tf.concat([x0, x1, x2, x3], 4)
    return outputs

In [ ]:
def I3D(inputs):
  x = Conv3D(64,
             (7,7,7),
             (2,2,2),
             activation='relu')(inputs)
  x = MaxPooling3D((3,3,3),
                   (2,2,2),
                   padding='same')(x)
  x = Conv3D(64,
             (1,1,1),
             activation='relu')(x)
  x = Conv3D(64,
             (3,3,3),
             activation='relu')(x)
  x = MaxPooling3D((3,3,3),
                   (2,2,2),
                   padding='same')(x)
  x = inc_block(x)
  x = inc_block(x)
  x = MaxPooling3D((3,3,3),
                   (2,2,2),
                   padding='same')(x)
  x = inc_block(x)
  x = inc_block(x)
  x = inc_block(x)
  x = inc_block(x)
  x = inc_block(x)
  x = MaxPooling3D((2,2,2),
                   (2,2,2),
                   padding='same')(x)
  x = inc_block(x)
  x = inc_block(x)
  x = AvgPool3D((2,7,7),
                strides=(1,1,1),
                padding='same')(x)
  x = Conv3D(64,
             (1,1,1))(x)
  x = Flatten()(x)
  x = Dense(128,
            activation='relu')(x)
  x = Dense(1,
            activation='sigmoid')(x)
  return x

In [ ]:
inputs = Input(shape=(40, 299, 299, 3))
outputs = I3D(inputs)

In [ ]:
inputs_i3d = Input(shape=(40,299,299,3))
i3d_output = I3D(inputs_i3d)

final_output = Add()([combined_output, i3d_output]) # addition +

combined_model = Model(inputs=[inputs, inputs_i3d], outputs=final_output)

In [ ]:
model = Model(inputs=inputs,
              outputs=outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit(X_train,
                    y_train,
                    batch_size=8,
                    validation_data = (X_val, y_val),
                    epochs=20,
                    callbacks=[tensorboard_callback])